In [ ]:
"""%pip install requests
   %pip install pandas"""

In [1]:
import requests
import pandas as pd

# Define the list of currencies you need
CURRENCIES = [
    'HUF', 'RON', 'CRC', 'TRY',
    'INR', 'PLN', 'THB', 'JPY',
    'CNY', 'BRL', 'HKD', 'CAD',
    'NOK', 'RUB', 'QAR', 'AED',
    'EGP', 'CHF', 'SGD', 'CZK',
    'DKK', 'KES', 'GBP', 'MXN',
    'TWD', 'CLP', 'KWD', 'USD',
    'ILS', 'NZD', 'KRW', 'SEK',
    'AMD', 'AUD', 'EUR'
]

def fetch_frankfurter_rates():
    """Fetch exchange rates from Frankfurter.app."""
    url = "https://api.frankfurter.app/latest?from=USD"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad status codes
        return response.json()['rates']
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from Frankfurter.app: {e}")
        return {}

def fetch_fallback_rates():
    """Fetch exchange rates from a fallback API (exchangerate-api.com)."""
    url = "https://open.er-api.com/v6/latest/USD"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()['rates']
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from fallback API: {e}")
        return {}

def get_exchange_rates():
    """Combine rates from Frankfurter.app and a fallback API."""
    # Fetch rates from Frankfurter.app
    frankfurter_rates = fetch_frankfurter_rates()
    
    # Fetch rates from the fallback API
    fallback_rates = fetch_fallback_rates()
    
    # Combine rates, prioritizing Frankfurter.app data
    rates = {}
    for currency in CURRENCIES:
        if currency in frankfurter_rates:
            rates[currency] = frankfurter_rates[currency]
        elif currency in fallback_rates:
            rates[currency] = fallback_rates[currency]
        else:
            print(f"Rate for {currency} not found in any API.")
            rates[currency] = None
    
    # Create a DataFrame
    df = pd.DataFrame([
        {'RatType': 'COBU', 'Currency': currency, 'RatCurrencyTo': 'USD', 'RatAmount': 1 / rate if rate else None}
        for currency, rate in rates.items()
    ])
    return df

if __name__ == "__main__":
    df = get_exchange_rates()
    if not df.empty:
        print(df)
        # Save the DataFrame to a CSV file
        df['RatDate'] = pd.Timestamp.now().strftime('%Y-%m-%d')
        df.to_csv('exchange_rates.csv', index=False)
    else:
        print("Failed to fetch exchange rates.")

   RatType Currency RatCurrencyTo  RatAmount
0     COBU      HUF           USD   0.002736
1     COBU      RON           USD   0.218933
2     COBU      CRC           USD   0.002003
3     COBU      TRY           USD   0.026321
4     COBU      INR           USD   0.011585
5     COBU      PLN           USD   0.260166
6     COBU      THB           USD   0.029740
7     COBU      JPY           USD   0.006674
8     COBU      CNY           USD   0.138311
9     COBU      BRL           USD   0.175960
10    COBU      HKD           USD   0.128694
11    COBU      CAD           USD   0.698275
12    COBU      NOK           USD   0.094715
13    COBU      RUB           USD   0.012211
14    COBU      QAR           USD   0.274725
15    COBU      AED           USD   0.272294
16    COBU      EGP           USD   0.019807
17    COBU      CHF           USD   1.137113
18    COBU      SGD           USD   0.750188
19    COBU      CZK           USD   0.043598
20    COBU      DKK           USD   0.146088
21    COBU